### RankNet-makedb-laptime

build laptime dataset from raw log files
functions of draw figures moved to PaperFigs

base: 18./laptime_rank_timediff_dataset

create laptime dataset only, no gluonts.

+ add lap2nextpit as target, this is as the same as a stacked lstm model when set prediction_length=1
+ add lap_instint to dynamic features(pitage)
+ add support for historical data, indy500 for different years
+ add track and lap status as target, preparing for StatusModel with deepAR

this notebook focuses on create the training set, while laptime2rank-evaluate will have testset codes specified for rank calculation

+ add new features, caution_laps_instint,lap_instint

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters

import indycar.model.global_variables as gvar
from indycar.model.quicktest_modules import *

register_matplotlib_converters()

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


## Load Data

In [2]:
outputRoot = 'data/'
os.makedirs(outputRoot, exist_ok=True)

stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0

#inlap status = 
# 0 , no inlap
# 1 , set previous lap
# 2 , set the next lap
_inlap_status = 0
_featureCnt = 9
#
# input data parameters
#
#events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events_totalmiles=[256,500,372,268,500,310]
#events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events_info = {
    'Phoenix':(256, 1.022, 250),'Indy500':(500,2.5,200),'Texas':(372,1.5,248),
    'Iowa':(268,0.894,300),'Pocono':(500,2.5,200),'Gateway':(310,1.25,248)
}

# event -> car#, maxlap
_race_info = {}

# the races have 7 years data 
races = ['Indy500', 'Texas','Iowa','Pocono']
years = ['2013','2014','2015','2016','2017','2018','2019']

#_race = 'Indy500'

events = []
for race in races:
    events.extend([f'{race}-{x}' for x in years])

#events.extend(['Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
#              'Gateway-2018','Gateway-2019'])
events.extend(['Phoenix-2018','Gateway-2018','Gateway-2019'])

events_id={key:idx for idx, key in enumerate(events)}

#version = f'IndyCar-d{len(events)}'
covergap = 1

#dbid = f'Indy500_{years[0]}_{years[-1]}_v{_featureCnt}_p{_inlap_status}'
dbid = f'IndyCar_d{len(events)}_v{_featureCnt}_p{_inlap_status}'

LAPTIME_DATASET = f'{outputRoot}/laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'{outputRoot}/stagedata-{dbid}.pickle' 
PITCOVERED_DATASET = f'{outputRoot}/pitcoveredlaps-{dbid}-g{covergap}.pickle' 

#
_test_race = 'xxx'
_test_event = 'Phoenix-2018'

In [3]:
gvar.events = events
gvar.events_id  = events_id
gvar.events_info = events_info


gvar.maxlap = get_event_info(_test_event)[2]
gvar.dbid = dbid
gvar.LAPTIME_DATASET = LAPTIME_DATASET

AttributeError: module 'indycar.model.global_variables' has no attribute '_race_info'

In [ ]:
for event in events:
    #dataid = f'{event}-{year}'
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event)

    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)-1))
    
    _race_info[event] = (len(carlist), len(laplist)-1, max(laplist))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1


In [ ]:
laptime_data = get_laptime_dataset(stagedata,inlap_status = _inlap_status)

In [ ]:
import pickle
#stintdf.to_csv('laptime-%s.csv'%year)
savefile = LAPTIME_DATASET
print(savefile)
with open(savefile, 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [global_carids, laptime_data]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
savefile = STAGE_DATASET 
print(savefile)
with open(savefile, 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [stagedata, _race_info, events, events_id]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

## Get pit-covered-laps

In [ ]:
### check the inlap
pitdata = {}
for event in events:    
    alldata, rankdata, acldata, flagdata = stagedata[event]

    totallaps = np.max(rankdata.completed_laps.to_numpy())
    #pitlaps = rankdata[rankdata['lap_status']=='P'][['completed_laps']].to_numpy()
    pitlaps = rankdata[rankdata['lap_status']=='P'].completed_laps.to_numpy()

    pitlaps = set(sorted(pitlaps))

    pitcoveredlaps = []
    for lap in pitlaps:
        gap = range(lap - covergap, lap + covergap+1)
        #pitcoveredlaps.extend([lap -2,lap-1,lap,lap+1,lap+2])
        pitcoveredlaps.extend(gap)

    pitcoveredlaps = set(sorted(pitcoveredlaps))
    
    print(event, 'total:', totallaps, 'pitlaps:', len(pitlaps), 'pitcoveredlaps:', len(pitcoveredlaps))
    
    #save
    pitdata[event] = [pitlaps, pitcoveredlaps]

In [ ]:
print('pitlaps:', len(pitdata[_test_event][0]), 'coveredlaps:', len(pitdata[_test_event][1]))

In [ ]:
import pickle
with open(PITCOVERED_DATASET, 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = pitdata
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

## test

In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Pocono-2019']
np.max(rankdata.completed_laps.values)

In [ ]:
_race_info